In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

- 상병기호 : ['E10', 'I10', 'J00', 'J20', 'M50']

In [3]:
df = pd.read_csv('./data/hackathon.csv')
df = df.iloc[:, 1:]
df.sample(15)

,시도코드,성별코드,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,ast_alt2,감마 지티피,상병기호
309809,41,2,23.875115,96.000000,95.0,238.0,91.0,17.4,1.0,1.0,0,23.0,0
143753,26,2,24.444444,115.333333,139.0,206.0,95.0,12.9,1.0,1.2,0,43.0,2
40068,43,2,26.666667,76.333333,109.0,118.0,91.0,13.3,2.0,0.6,0,17.0,3
134409,41,1,22.491349,95.000000,101.0,222.0,112.0,16.3,1.0,0.9,0,23.0,3
39489,48,2,19.531250,96.666667,80.0,202.0,149.0,13.1,1.0,0.8,0,16.0,4
78393,29,2,20.811655,83.000000,84.0,176.0,83.0,13.0,1.0,0.7,0,12.0,4
79124,11,2,27.055151,98.000000,88.0,182.0,60.0,12.6,1.0,0.6,0,23.0,2
88523,45,2,20.000000,94.333333,84.0,149.0,48.0,13.7,1.0,0.7,0,10.0,4
133511,41,1,32.653061,102.333333,92.0,161.0,85.0,14.5,1.0,0.9,0,20.0,2
278993,11,2,27.055151,92.666667,94.0,169.0,98.0,13.7,1.0,0.8,0,16.0,2


In [4]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# 데이터에 LabelEncoder 적용
encoded_data = label_encoder.fit_transform(df['시도코드'])
df['시도코드'] = encoded_data

# 전처리

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   시도코드        330958 non-null  int64  
 1   성별코드        330958 non-null  int64  
 2   bmi         330958 non-null  float64
 3   map         330958 non-null  float64
 4   식전혈당(공복혈당)  330958 non-null  float64
 5   총 콜레스테롤     330958 non-null  float64
 6   트리글리세라이드    330958 non-null  float64
 7   혈색소         330958 non-null  float64
 8   요단백         330958 non-null  float64
 9   혈청크레아티닌     330958 non-null  float64
 10  ast_alt2    330958 non-null  int64  
 11  감마 지티피      330958 non-null  float64
 12  상병기호        330958 non-null  int64  
dtypes: float64(9), int64(4)
memory usage: 32.8 MB


In [6]:
temp = df.select_dtypes(include=['int64']).astype('category')

temp2 = df.select_dtypes(include=['float64']).astype('float32')

scaler = StandardScaler()
temp2 = pd.DataFrame(scaler.fit_transform(temp2), columns=temp2.columns) #np -> pd

df2 = pd.concat([temp2, temp], axis=1)
df2.info()
display(df2[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   bmi         330958 non-null  float32 
 1   map         330958 non-null  float32 
 2   식전혈당(공복혈당)  330958 non-null  float32 
 3   총 콜레스테롤     330958 non-null  float32 
 4   트리글리세라이드    330958 non-null  float32 
 5   혈색소         330958 non-null  float32 
 6   요단백         330958 non-null  float32 
 7   혈청크레아티닌     330958 non-null  float32 
 8   감마 지티피      330958 non-null  float32 
 9   시도코드        330958 non-null  category
 10  성별코드        330958 non-null  category
 11  ast_alt2    330958 non-null  category
 12  상병기호        330958 non-null  category
dtypes: category(4), float32(9)
memory usage: 12.6 MB


,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,감마 지티피,시도코드,성별코드,ast_alt2,상병기호
0,0.182520,-0.621876,-0.717914,0.831830,0.499025,1.426068,-0.253558,0.764726,-0.219147,2,1,0,1
1,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,11,1,0,2
2,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,11,1,0,3
3,-0.549760,-0.373717,-0.003174,-0.503164,-0.735804,-0.559361,-0.253558,-0.443400,0.033405,8,2,0,2
4,-0.361858,-0.311677,-0.479667,-1.088688,0.418053,-0.751499,-0.253558,-0.443400,-0.491125,8,2,0,0
5,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,5,1,0,3
6,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,5,1,0,2
7,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,11,2,0,3
8,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,11,2,0,2
9,-0.944999,-0.559836,-0.400252,1.300249,-0.492887,-1.520052,-0.253558,-0.141369,-0.491125,8,2,0,0


# XGB

In [7]:
train_input, test_input, train_target, test_target = train_test_split(df2.iloc[:, :-1], df2.iloc[:,-1], test_size=0.2, random_state=42)

In [8]:
model = XGBClassifier(
    objective='softprob',  # 다중분류
    num_class=5,  # 클래스 개수 설정
    enable_categorical=True # 범주형 변수 유지
)

# 모델 학습
model.fit(train_input, train_target)

# 예측 수행
y_pred = model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.30337503021513174


In [9]:
confusion_matrix(test_target, y_pred)

array([[    0,    23,  5275,   357,    34],
       [   10,    13,  9776,   724,    60],
       [   13,    70, 19204,  1812,   124],
       [    6,    47, 16165,   846,    82],
       [    6,    50, 10684,   793,    18]], dtype=int64)

# 튜닝_RFE

In [10]:
model = XGBClassifier(
    objective='multi:softmax',  # 다중분류
    num_class=5,  # 클래스 개수 설정
    enable_categorical=True, # 범주형 변수 유지
    max_depth = 5
)

# RFE를 사용하여 변수 선택
rfe = RFE(model, n_features_to_select=9)  # 선택할 특성의 개수를 지정합니다.

# 변수 선택 및 학습용 데이터 변환
X_train_rfe = rfe.fit_transform(train_input, train_target)
X_test_rfe = rfe.transform(test_input)

# 모델 학습
model.fit(X_train_rfe, train_target)

# 예측 수행
y_pred = model.predict(X_test_rfe)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.31222806381435825


In [11]:
confusion_matrix(test_target, y_pred)

array([[    1,     8,  5506,   163,    11],
       [    4,     6, 10244,   311,    18],
       [    2,    22, 20268,   888,    43],
       [    1,    15, 16713,   388,    29],
       [    7,    14, 11197,   329,     4]], dtype=int64)

In [12]:
df['상병기호'].value_counts()/len(df)

2    0.320047
3    0.260819
4    0.175651
1    0.157603
0    0.085881
Name: 상병기호, dtype: float64